In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

2023-04-12 15:18:27.823547: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-12 15:18:27.823623: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import tensorflow as tf
from keras.layers import Dense , Activation , Dropout, Embedding, SpatialDropout1D, LSTM , TextVectorization, TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [3]:
from bs4 import BeautifulSoup
import os
import re

In [4]:
directory = 'corpus_morphological_analysis'
file_paths = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    file_paths.append(f)    

In [5]:
temp_file_paths = file_paths[:100]

In [6]:
content = []
#i = 0
for filepath in temp_file_paths :
    #print(i)
    with open(filepath, encoding='utf-8') as f :
        html = f.read()
    soup = BeautifulSoup(html, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()  
    text = soup.get_text()
    content.append(text)
    #i+=1

In [7]:
content[0]

'مسقط:مُسْقَطٍ:#:مسقط:اسم زمان أو مكان:مُفْعَلٍ:سقط:#:\nالعمانية: لا توجد نتائج لتحليل هذه الكلمة\nيقام:يُقَامَ:ي: حرف المضارعة:يقام:فعل مضارع مبني للمجهول:يُفاعَ:قمي:#:\nفى: لا توجد نتائج لتحليل هذه الكلمة\nالرابع:الرَّابِعِ:ال: التعريف:رابع:اسم فاعل:فَاعِلِ:ربع:#:\nوالعشرين:وَالْعُشْرَيْنِ:وَال: حرف العطف+ال التعريف:عشرين:اسم جامد:فُعْلَيْنِ:عشر:ين:علامة الإعراب:\nمن:مَنْ:# :من:اسم شَرْط:#:#:# :\nشهر:شَهِّرْ:#:شهر:فعل أمر:فَعِّلْ:شهر:#:\nفبراير:فَبِرَايِرٍ:فَبِ: حرف العطف+حرف الجر:راير:اسم فاعل:فَاعِلٍ:رير:#:\nالمقبل:الْمِقْبَلِ:ال: التعريف:مقبل:اسم آلة:مِفْعَلِ:قبل:#:\nبفندق: لا توجد نتائج لتحليل هذه الكلمة\nقصر:قَصِّرْ:#:قصر:فعل أمر:فَعِّلْ:قصر:#:\nالبستان:الْبَسَّتَانِ:ال: التعريف:بستان:مصدر مرة:فَعَّتَانِ:بسس:ة:تاء التأنيث+ا علامة الإعراب:\nالمؤتمر:الْمُؤْتَمَرِ:ال: التعريف:مؤتمر:اسم زمان أو مكان:مُفْتَعَلِ:ءمر:#:\nالدولى: لا توجد نتائج لتحليل هذه الكلمة\nلفرص:لَفَرِّصْ:لَ: حرف الابتداء:فرص:فعل أمر:فَعِّلْ:فرص:#:\nالاستثمار:الْاسْتِثْمَارِ:ال: التعريف:استثمار:مصدر أصلي:اسْتِفْعَا

In [8]:
len(content)

100

In [9]:
split_list = []
for item in content : 
    tmp = item.splitlines()
    split_list.append(tmp)

In [10]:
work_list = []
for k in split_list :
    l = [item for item in k if 'لا توجد نتائج لتحليل هذه الكلمة' not in item]
    tmp_l = [item.replace("#",'') for item in l]
    work_list.append(tmp_l)

In [11]:
final_list = []
for k in work_list :
    tst = [item.split(':') for item in k]
    final_list.append(tst)

In [12]:
# identifie le prefixe, racine et le suffixe et les placent dans un dictionnaire
def identify(word_l):
    if len(word_l) < 4 :
        return None
    dictt = {}
    dictt['word'] = word_l[1]
    # le cas s'il existe un préfixe
    if word_l[2] != '' and word_l[2] != ' ' :   
        if word_l[4] not in word_l[0]: 
            if word_l[5] in word_l[0] and word_l[5] != '': 
                dictt['prefixe'] = word_l[2]
                dictt['root'] = word_l[8]
                dictt['suffixe'] = word_l[9]
            elif word_l[3] in word_l[0] and word_l[3] != '':
                dictt['prefixe'] = word_l[2]
                dictt['root'] = word_l[3]
                dictt['suffixe'] = ''
        else :
            dictt['prefixe'] = word_l[2]
            dictt['root'] = word_l[7]
            dictt['suffixe'] = word_l[8]
    # s'il n'existe pas un préfixe
    else : 
        if word_l[2] == '' : 
            dictt['prefixe'] = word_l[2]
            dictt['root'] = word_l[6]
            dictt['suffixe'] = word_l[7]
        elif  word_l[2] == ' ' :
            dictt['prefixe'] = ''
            dictt['root'] = word_l[3]
            dictt['suffixe'] = ''    
    return dictt

In [13]:
# filtre la liste de mots en liste de dictionnaires
def word_to_dict_list(wordlist):
    dictlist = []
    for k in wordlist : 
        dictlist.append(identify(k))
    return dictlist

In [14]:
final = []
for k in final_list: 
    for j in k :
        s = identify(j)
        if s == None :
            continue
        final.append(identify(j))

In [15]:
print(final[1000])

{'word': 'الْجَاهِزَةِ', 'prefixe': 'ال', 'root': 'جهز', 'suffixe': 'ة'}


In [16]:
def dic_to_list(listt):
    L = []
    for k in listt : 
        tmp = []
        #print(k)
        if len(k) == 4 : 
            tmp.append(k['word'])
            tmp.append(k['prefixe'])
            tmp.append(k['root'])
            tmp.append(k['suffixe'])
            L.append(tmp)
    return L
data = dic_to_list(final)

In [17]:
final_l = dic_to_list(final)
final_l[0]

['مُسْقَطٍ', '', 'سقط', '']

In [18]:
# experimenting with our first neural network to predict the root of a word.

data_1 = []

for word in final_l:
    tmp = []
    word[0] = word[0].replace('+', ' ')
    word[2] = word[2].replace('+', ' ')
    tmp.append(word[0])
    tmp.append(word[2])
    data_1.append(tmp)
    #print(tmp)
data_1

[['مُسْقَطٍ', 'سقط'],
 ['يُقَامَ', 'قمي'],
 ['الرَّابِعِ', 'ربع'],
 ['وَالْعُشْرَيْنِ', 'عشر'],
 ['مَنْ', 'من'],
 ['شَهِّرْ', 'شهر'],
 ['فَبِرَايِرٍ', 'رير'],
 ['الْمِقْبَلِ', 'قبل'],
 ['قَصِّرْ', 'قصر'],
 ['الْبَسَّتَانِ', 'بسس'],
 ['الْمُؤْتَمَرِ', 'ءمر'],
 ['لَفَرِّصْ', 'فرص'],
 ['الْاسْتِثْمَارِ', 'ثمر'],
 ['وَالتَّجَّارَةِ', 'تجر'],
 ['تَنَظَّمْهُ', 'نظم'],
 ['مِجْلَةً', 'مجل'],
 ['مِيدَ', 'ميد'],
 ['الْمِجْلَةَ', 'مجل'],
 ['لِلشَّرْقِ', 'شرق'],
 ['بِالتَّعَاوُنِ', 'عون'],
 ['مَعَ', 'مع'],
 ['الْمِرْكَزِ', 'ركز'],
 ['لِتَرْوِيجٍ', 'روج'],
 ['الْاسْتِثْمَارِ', 'ثمر'],
 ['وَتَنَمِّيَةٍ', 'نمي'],
 ['الصَّادِرَاتِ', 'صدر'],
 ['وَيُسْتَمَرَّ', 'مرر'],
 ['لَمِدَّةً', 'مدد'],
 ['يَوْمَيْنِ', 'يوم'],
 ['وَيُشْتَمَلْ', 'شمل'],
 ['الْمُؤْتَمَرِ', 'ءمر'],
 ['تُحْتَ', 'حتو'],
 ['عِنْوَانٍ', 'عنون'],
 ['الْفَرْصِ', 'فرص'],
 ['الْكَبِيرَةِ', 'كبر'],
 ['وَالْاسْتِثْمَارَاتِ', 'ثمر'],
 ['السَّلْطَنَةِ', 'سلطن'],
 ['عَلَى', 'على'],
 ['وَصُفْ', 'صوف'],
 ['شَامِلٍ', 'شمل'],
 ['لِلْاقْتِصَادِ', 'قصد'

In [19]:
# we extract our dictionnary from the our dataset 
def extract_dict(listt) :
    dictt = []
    for word in listt :
        for item in word : 
            tmp = set(item)
            for k in tmp : 
                if k not in dictt : 
                    dictt.append(k)
    return dictt       
dic = extract_dict(data_1)

In [20]:
root_data = []
for word in data : 
    tmp =[]
    tmp.append(word[0])
    tmp.append(word[2])
    root_data.append(tmp)
root_data

[['مُسْقَطٍ', 'سقط'],
 ['يُقَامَ', 'قمي'],
 ['الرَّابِعِ', 'ربع'],
 ['وَالْعُشْرَيْنِ', 'عشر'],
 ['مَنْ', 'من'],
 ['شَهِّرْ', 'شهر'],
 ['فَبِرَايِرٍ', 'رير'],
 ['الْمِقْبَلِ', 'قبل'],
 ['قَصِّرْ', 'قصر'],
 ['الْبَسَّتَانِ', 'بسس'],
 ['الْمُؤْتَمَرِ', 'ءمر'],
 ['لَفَرِّصْ', 'فرص'],
 ['الْاسْتِثْمَارِ', 'ثمر'],
 ['وَالتَّجَّارَةِ', 'تجر'],
 ['تَنَظَّمْهُ', 'نظم'],
 ['مِجْلَةً', 'مجل'],
 ['مِيدَ', 'ميد'],
 ['الْمِجْلَةَ', 'مجل'],
 ['لِلشَّرْقِ', 'شرق'],
 ['بِالتَّعَاوُنِ', 'عون'],
 ['مَعَ', 'مع'],
 ['الْمِرْكَزِ', 'ركز'],
 ['لِتَرْوِيجٍ', 'روج'],
 ['الْاسْتِثْمَارِ', 'ثمر'],
 ['وَتَنَمِّيَةٍ', 'نمي'],
 ['الصَّادِرَاتِ', 'صدر'],
 ['وَيُسْتَمَرَّ', 'مرر'],
 ['لَمِدَّةً', 'مدد'],
 ['يَوْمَيْنِ', 'يوم'],
 ['وَيُشْتَمَلْ', 'شمل'],
 ['الْمُؤْتَمَرِ', 'ءمر'],
 ['تُحْتَ', 'حتو'],
 ['عِنْوَانٍ', 'عنون'],
 ['الْفَرْصِ', 'فرص'],
 ['الْكَبِيرَةِ', 'كبر'],
 ['وَالْاسْتِثْمَارَاتِ', 'ثمر'],
 ['السَّلْطَنَةِ', 'سلطن'],
 ['عَلَى', 'على'],
 ['وَصُفْ', 'صوف'],
 ['شَامِلٍ', 'شمل'],
 ['لِلْاقْتِصَادِ', 'قصد'

In [21]:
class encode :
    
    def __init__(self, data):
        self.data = data
        
    def code_sequence(self):
        L = []
        for word in self.data : 
            tmp_string = code_word(word)
            L.append(tmp_string)
        final_string = '#'.join(map(str,L))
        return final_string 
    
    def code_normal_text(self):
        L = []
        for word in self.data : 
            L.append(word[1])
        final_string = ' '.join(map(str,L))
        return final_string
        
final_str = encode(root_data).code_normal_text()      

In [22]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Define the GRU layer
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        
        # Define the output layer
        self.out = nn.Linear(hidden_size, output_size)
        
        # Define the softmax activation function
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, e, hsowi):
        """
        e: Encoder representation of the input word
        hsowi: Start-of-word character
        
        Returns the predicted root word and the attention weights
        """
        
        # Initialize the hidden state h0
        h0 = torch.zeros(1, e.size(0), self.hidden_size)
        
        # Concatenate the encoder representation and start-of-word character
        x = torch.cat([hsowi.unsqueeze(1), e], dim=1)
        
        # Apply the GRU layer
        out, h = self.gru(x, h0)
        
        # Apply the output layer
        out = self.out(out)
        
        # Apply the softmax activation function
        out = self.softmax(out)
        
        # Return the predicted root word and the attention weights
        return out


In [74]:
# implementation of a paper : 

import tensorflow as tf
import numpy as np
import torch.nn as nn
import torch 
import string


class model : 
    
    def __init__(self, data, embedding_size):
        '''
        our input data are arabic words with their roots, with the hypothesis that each word has it's own root.
        
        (our dataset for this is named root_data)
        '''
        self.data = data
        self.embedding_size = embedding_size
        self.vocab = self.extract_dict()
        self.char_index_dic = self.char_to_index() 
        #self.hsowi = torch.tensor(self.char_index_dic[hsowi]).unsqueeze(0)

    
    def extract_dict(self) :
        '''
        this function extracts all the unique characters from the given dataset
        '''
        dictt = []
        for word in self.data :
            for item in word : 
                tmp = set(item)
                for k in tmp : 
                    if k not in dictt : 
                        dictt.append(k)
        return dictt 
    
    
    # Let's now construct our model : 
    
    # we should think about character embeddings in order to create an embeded matrix for each word
    
    def char_to_index(self):
        '''
        this function creates unique indexes of each character
        '''
        char_to_idx_map = {char: idx for idx, char in enumerate(self.vocab)}
        #print(f"Character to Index Mapping:\n{char_to_idx_map}\n")
        return char_to_idx_map  
    
    
    def word_to_idx_seq(self, word):
        '''
        this function returns a sequence of the unique indexes for the given word 
        (sequence is tensor that can be changed using a .tolist() )
        '''
        word_char_idx_tensor = torch.tensor([self.char_index_dic[char] for char in word])        
        return word_char_idx_tensor # word sequence
    
    
    def embedding(self, word_sequence):    
        '''
        this function creates a character based embedding of each word as a sequence of characters 
        '''
        embedding_layer = nn.Embedding(num_embeddings = len(self.vocab), embedding_dim = self.embedding_size) 
        embedding_vector = embedding_layer(word_sequence)
        return embedding_vector
    
    
    def encode_word(self, word):
        
        hidden_size = 32  # Number of hidden units in the GRU
        bidirectional = True  # Whether to use a bidirectional GRU or not
        num_layers = 1  # Number of layers in the GRU
        gru = nn.GRU(input_size=self.embedding_size, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, batch_first=True)
        
        #@jit update_function
        
        word_seq = self.word_to_idx_seq(word)
        embedded_vec = self.embedding(word_seq)
        #print(embedded_vec.size())
        outputs, hidden = gru(embedded_vec)
        
        """print(word)
        print(hidden.size())
        print(hidden[-2].size())
        print(hidden[-1].size())"""
        
        # Concatenate the forward and backward hidden vectors        
        h1 = torch.reshape(hidden[-1], (1,hidden_size))
        h2 = torch.reshape(hidden[-2], (1,hidden_size))
        hidden = torch.cat((h2, h1),1)  # Shape: (batch_size, hidden_size*2)
        
        encoder_output = torch.mean(hidden, dim=0)  # Average the hidden vectors across all time steps. Shape: (hidden_size*2,) if bidirectional, else (hidden_size,)
     
        return encoder_output
    
    
    def decode_word(self, encoding):

        num_layers = 1  # Number of layers in the GRU
        hidden_size= 32 # Number of hidden elements in the GRU
        input_size = encoding.size(0) + 1
        gru = nn.GRU(input_size= input_size ,hidden_size = hidden_size, num_layers = num_layers) 
        
        loss_fn = nn.CrossEntropyLoss()

        
        
        
        return 

    
    
    def main(self):
        
        test = 0
        for instance in self.data : 
            word = instance[0]
            test = self.encode_word(word)
            size = test.size(0)
            #out = self.decode_word(test)
            
            #print(word)
            #print(out)
            break
            
            
            
        return test    




In [75]:
mod = model(root_data, 10)
mod.main()

64


tensor([ 0.1097, -0.2494,  0.4551, -0.2999,  0.0094, -0.1662, -0.0855, -0.0695,
         0.1338, -0.2104, -0.2010, -0.0823,  0.4442,  0.1741,  0.1659,  0.0608,
        -0.3120,  0.1714,  0.0645,  0.1078,  0.3804, -0.4028,  0.0044,  0.2451,
        -0.3783, -0.2358, -0.2026,  0.0716,  0.1852, -0.4934,  0.3521,  0.0986,
         0.0717,  0.0531,  0.1596,  0.1790,  0.0125,  0.1555,  0.0510,  0.2361,
         0.2106,  0.0768,  0.0861,  0.2359, -0.2281, -0.0086,  0.1196,  0.1971,
         0.0827, -0.1407,  0.1999,  0.3002, -0.3455,  0.0017,  0.0266,  0.3917,
         0.0902,  0.0425,  0.1267, -0.0176, -0.3630,  0.0590,  0.3767,  0.2752],
       grad_fn=<MeanBackward1>)

In [73]:
l = 0
for item in root_data:
    if len(item[1]) > 3:
        print(item[1])

عنون
سلطن
جديدة
سلطن
خميس
هندس
هندس
سلطن
سلطن
سلطن
سلطن
سلطن
سلطن
التي
التي
التي
التي
التي
سلطن
جمهر
رديف
رسول
صلهم
الذي
الذي
جبريل
فيما
الذي
الذي
رسول
بينما
جلجل
رسول
حمود
عمانية
هذين
هذين
التي
عواطف
الذي
خميس
ذياب
ترجم
التي
سبتمبر
سلطن
رفرف
الذي
التي
التي
ترجم
كاميليا
الذي
يونيو
كاميليا
عطية
كاميليا
التي
بينما
هوية
بابه
فيما
التي
التي
سلطن
سلطن
سلطن
التي
برمج
سلطن
سلطن
سلطن
رحيلة
الذي
سلطن
التي
آذان
الذي
الذي
التي
التي
الذي
فبراير
التي
سبتمبر
الذي
دهور
فبراير
التي
الذي
بينما
الذي
أمريكي
مايكل
بابه
عندما
الذي
موسكو
يوليو
سبتمبر
سيطر
ديفيد
سيطر
عندما
التي
كامبل
التي
جونز
التي
التي
الذي
التي
سيطر
الذي
التي
موسكو
عزيز
خرطم
مصطفى
زعيم
قرنق
مصطفى
مصطفى
هكذا
الذي
التي
قبيل
مصطفى
الذين
بينما
مصطفى
الذي
التي
مصطفى
التي
مصطفى
مصطفى
التي
جعفر
نميري
قرنق
قرنق
الذي
قرنق
مونتريال
عنون
سلطن
سلطن
التي
سلطن
جمهر
التي
سلطن
التي
التي
التي
التي
مهما
مهما
التي
التي
التي
التي
فيما
التي
سلطن
التي
التي
سلطن
سلطن
التي
سلطن
الذي
فيما
الذين
جدوى
شقشق
التي
الذين
التي
التي
جديدة
هيمن
الذي
جديدة
التي
هؤلاء
التي
ا

In [24]:
'''def decode_word(self, word):
        
        hidden_size = 32  # Number of hidden units in the GRU
        num_layers = 1  # Number of layers in the GRU
        dropout = 0.2  # Dropout rate
        
        gru = nn.GRU(input_size=self.embedding_size, hidden_size=hidden_size, batch_first=True)
        
        e = self.encode_word(word)
        # Initialize the hidden state h0
        
        h0 = torch.zeros(1, hidden_size)

        
        # Concatenate the encoder representation and start-of-word character
        print(e.size())
        print(self.hsowi.unsqueeze(1).size())
        
        x = torch.cat([self.hsowi.unsqueeze(0).float(), e.unsqueeze(0)], dim=1)

        print(x.size())
        print(h0.size())
        
        # Apply the GRU layer
        out, h = gru(x)
        
        # Apply the output layer
        out_l = nn.Linear(hidden_size, len(self.vocab))
        
        out = out_l(out)
        
        softmax = nn.Softmax(dim=2)
        
        # Apply the softmax activation function
        out = softmax(out)
        
        return out
    '''

'def decode_word(self, word):\n        \n        hidden_size = 32  # Number of hidden units in the GRU\n        num_layers = 1  # Number of layers in the GRU\n        dropout = 0.2  # Dropout rate\n        \n        gru = nn.GRU(input_size=self.embedding_size, hidden_size=hidden_size, batch_first=True)\n        \n        e = self.encode_word(word)\n        # Initialize the hidden state h0\n        \n        h0 = torch.zeros(1, hidden_size)\n\n        \n        # Concatenate the encoder representation and start-of-word character\n        print(e.size())\n        print(self.hsowi.unsqueeze(1).size())\n        \n        x = torch.cat([self.hsowi.unsqueeze(0).float(), e.unsqueeze(0)], dim=1)\n\n        print(x.size())\n        print(h0.size())\n        \n        # Apply the GRU layer\n        out, h = gru(x)\n        \n        # Apply the output layer\n        out_l = nn.Linear(hidden_size, len(self.vocab))\n        \n        out = out_l(out)\n        \n        softmax = nn.Softmax(dim=2)\

In [25]:
data_root = []
for item in root_data : 
    tmp = []
    tmp.append(item[0])
    tmp.append('$'+item[1])
    data_root.append(tmp)

In [ ]:
import tensorflow as tf

# create an embedding layer with 50 input dimensions and 100 output dimensions
embedding_layer = tf.keras.layers.Embedding(input_dim=43, output_dim=100)
sequence = [5 , 10 , 10, 20 , 7, 1]
# get the embedding vectors for the sequence of character indices
embedding_vectors = embedding_layer(np.array(sequence))
embedding_vectors

In [ ]:
hidden_size = 32  # Number of hidden units in the GRU
bidirectional = True  # Whether to use a bidirectional GRU or not
num_layers = 1  # Number of layers in the GRU
dropout = 0.2  # Dropout rate
gru = nn.GRU(input_size=16, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)


In [ ]:
import torch
import torch.nn as nn
import string

In [ ]:
char_to_idx_map = {char: idx for idx, char in enumerate(dic)}
print(f"Character to Index Mapping:\n{char_to_idx_map}\n")

In [ ]:
ohe_characters = torch.eye(n=len(dic))  # using the eye method for identity matrix
print(f"One hot encoded characters:\n{ohe_characters}\n")

In [ ]:
ohe_characters.shape

In [ ]:
ohe_repr_a = ohe_characters[char_to_idx_map['ح']]
print(f"One hot encoded representation of 1:\n{ohe_repr_a}\n")

In [ ]:
def encode_word(word, char_to_idx_map):
    
    word_char_idx_tensor = torch.tensor([char_to_idx_map[char] for char in word])
    word_ohe_repr = ohe_characters[word_char_idx_tensor].T
    
    return word_ohe_repr

In [ ]:
word_char_idx_tensor = torch.tensor([char_to_idx_map[char] for char in "مَرْحَبًا"])
print(word_char_idx_tensor.tolist())
word_ohe_repr = ohe_characters[word_char_idx_tensor].T
word_ohe_repr.shape

In [ ]:
convolution_layer = nn.Conv1d(in_channels=43, out_channels=128, kernel_size=3, bias=False)
conv_out = convolution_layer(word_ohe_repr.unsqueeze(0))

In [ ]:
print(f"Shape of convolution output: {conv_out.shape}")
print(f"Convolution Output:\n{conv_out}\n")

# adding an activation layer so as to add non linearity to the output
activation_layer = nn.Tanh()
activation_out = activation_layer(conv_out)
activation_out

In [ ]:
print(f"Shape of activation output: {activation_out.shape}")
print(f"Activation Output:\n{activation_out}")

In [ ]:
# Max Pooling Layer
max_pooling_layer = nn.MaxPool1d(kernel_size=conv_out.shape[2])

# finally here is our word embedding of size -> 256
word_embedding = max_pooling_layer(activation_out).squeeze()

print(f"Final word embedding shape: {word_embedding.shape}")
print(f"Final word embedding:\n{word_embedding}")

In [ ]:
def character_embedding():
    

### Let's create a data structure for each word of the corpus

In [ ]:
tmp = work_list[2]
tmp.split(":")

In [ ]:
vowel_pattern = r'[\u064b\u064e\u064f\u0650\u0651\u0652]'
word = "مَرْحَبًا"
vowels = re.findall(vowel_pattern, word)
print(f"Vowels in '{word}': {vowels}")

In [ ]:
'''filtered_split_list = []
for item in split_list : 
    temp_item_list = []
    for subitem in item : 
        if 'لا توجد نتائج لتحليل هذه الكلمة' not in item : 
            temp_item_list.append(item)
    filtered_split_list.append(temp_item_list)
filtered_split_list[0]'''

In [ ]:
'''# Check the data type of each element in texts
texts = pd.DataFrame(data_1[:, -1])
texts = texts.iloc[:, 1:]
for i, text in enumerate(texts):
    if not isinstance(text, str):
        print(f"Element {i} is not a string: {text}")'''